In [1]:
import pandas as pd
import re
from datetime import datetime
from dateutil.parser import parse
import missingno as msno
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import json

In [10]:
inspect_df = pd.read_csv('2015-2020-apps-intl.csv')

In [17]:
inspect_df.dtypes

STU_ID                      object
BDAY                datetime64[ns]
COLLEGE                     object
DEGREE                      object
MAJOR                       object
LEGAL_NATN                  object
STREET_LINES_1_2            object
CITY_STATE_ZIP              object
ZIP                         object
dtype: object

In [15]:
def dateobj(str):
    return datetime.strptime(str, '%m/%d/%Y')

In [16]:
inspect_df['BDAY'] = inspect_df['BDAY'].apply(dateobj)

In [18]:
inspect_df.head()

,STU_ID,BDAY,COLLEGE,DEGREE,MAJOR,LEGAL_NATN,STREET_LINES_1_2,CITY_STATE_ZIP,ZIP
0,A04188656,1997-03-15,SE,MS,COSC,India,7350 McArdle Rd Apt 86,"Corpus Christi, TX 78412-4234",78412-4234
1,A04151230,1992-09-01,BU,MBA,BUSI,Pakistan,House 3 51 Street 35 F-6/1,"Islamabad, 44000",44000
2,A04151230,1992-09-01,BU,MBA,BUSI,Pakistan,House 3 51 Street 35 F-6/1,"Islamabad, 44000",44000
3,A04051997,1986-12-01,SE,MS,COSC,Iran,"No.38, Farimah Str.,Hellal Ahmar Sq Jahanshahr","Karaj, 3144753838",3144753838
4,A03988744,1987-06-14,SE,MS,COSC,India,"#2005, 8th Main, 2nd Stage Kumarswamy Layout","Bangalore, 560078",560078


In [19]:
inspect_df.shape

(3763, 9)

In [20]:
inspect_places = inspect_df['STU_ID'].unique()

In [75]:
# those are counts for all US
inspect_df['LEGAL_NATN'].value_counts().nlargest(31)
producers_df = inspect_df['LEGAL_NATN'].value_counts().rename_axis('country').reset_index(name='count')
producers_df.head(15)

,country,count
0,India,1682
1,Vietnam,379
2,Nigeria,271
3,China,182
4,Saudi Arabia,126
5,Mexico,102
6,Pakistan,79
7,Canada,65
8,Iran,50
9,Colombia,42


In [60]:
# those are counts for all US
producers_df.tail()

,country,count
104,Portugal,1
105,Senegal,1
106,French Southern/Antarct,1
107,Rwanda,1
108,Bahamas,1


In [61]:
# all US
inspect_df.loc[1]

STU_ID                                A04151230
BDAY                        1992-09-01 00:00:00
COLLEGE                                      BU
DEGREE                                      MBA
MAJOR                                      BUSI
LEGAL_NATN                             Pakistan
STREET_LINES_1_2    House 3 51 Street 35 F-6/1 
CITY_STATE_ZIP                 Islamabad, 44000
ZIP                                       44000
Name: 1, dtype: object

In [46]:
inspect_df.groupby(['MAJOR','LEGAL_NATN'])['COLLEGE'].count().nlargest(30)


MAJOR  LEGAL_NATN  
COSC   India           1337
BUSI   Vietnam          196
BNAP   India            130
ACCT   Vietnam           63
BNAP   Nigeria           61
BUSI   Saudi Arabia      57
       India             46
BNAP   Mexico            45
       Canada            40
COSC   Nigeria           37
ACCT   China             35
BUAD   Nigeria           33
       India             32
BUSI   China             32
COSC   Vietnam           31
BNAP   Pakistan          26
BUAD   Vietnam           25
COSC   China             25
BUSI   Nigeria           24
KINE   India             22
PADM   Nigeria           22
GSCS   China             21
BNAP   Philippines       20
GSCS   Iran              19
COSC   Pakistan          18
COMM   Nigeria           17
ACCT   Saudi Arabia      16
ESCI   India             14
ACCT   Nigeria           13
BNAP   Colombia          13
Name: COLLEGE, dtype: int64

In [70]:
import numpy as np
import folium

# create our own threshold_scale
threshold_scale = np.linspace(producers_df['count'].min(), producers_df['count'].max(), 6, dtype=int)
# convert the numpy array into a list
threshold_scale = threshold_scale.tolist() 
# increase one unit the last value of the list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# file name - file is located in the working directory
world_geo = r'world-countries.json' # geojson file

# use this threshold_scale to create the choropleth map.
m2 = folium.Map(location=[0, 0], zoom_start=2)
m2.choropleth(
    geo_data=world_geo,
    data=producers_df,
    columns=['country', 'count'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
#    fill_color='YlGnBu',
    fill_color='RdYlGn',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Number of applicants'
)
folium.LayerControl().add_to(m2)
m2.save(outfile='world_apps_map.html')




In [74]:
import numpy as np
import folium

producers_filtered_df = producers_df.query('country != "India"')

# create our own threshold_scale
threshold_scale = np.linspace(producers_filtered_df['count'].min(), producers_filtered_df['count'].max(), 6, dtype=int)
# convert the numpy array into a list
threshold_scale = threshold_scale.tolist() 
# increase one unit the last value of the list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# file name - file is located in the working directory
world_geo = r'world-countries.json' # geojson file

# use this threshold_scale to create the choropleth map.
m3 = folium.Map(location=[0, 0], zoom_start=2)
m3.choropleth(
    geo_data=world_geo,
    data=producers_filtered_df,
    columns=['country', 'count'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
#    fill_color='YlGnBu',
    fill_color='RdYlGn',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Number of applicants'
)
folium.LayerControl().add_to(m3)
m3.save(outfile='world-minus-india_apps_map.html')


In [100]:
# PREPADRE GEO data

# load GeoJSON
with open('tx_texas_zip_codes_geo.min.json', 'r') as jsonFile:
    data = json.load(jsonFile)
tmp = data

# remove ZIP codes not in our dataset
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['ZCTA5CE10'] in list(geo_agg['ZIP'].unique()):
        geozips.append(tmp['features'][i])

# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("updated-tx-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)

205520749

In [81]:
import folium

# table = main table/data frame we read from (pandas DataFrame)
# zips = column name where ZIP codes are (string)
# mapped_feature = column name for feature we want to visualize (string)
# add_text = any additional commentary to be added in the map legend (string)

def create_map(table, zips, mapped_feature, add_text = ''):
    # reading of the updated GeoJSON file
    la_geo = r'updated-file.json'
    # initiating a Folium map with LA's longitude and latitude
    m = folium.Map(location = [27.8006, -97.3964], zoom_start = 7)
    # creating a choropleth map
    m.choropleth(
        geo_data = la_geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'RdYlGn',
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' Across LA'
    )
    folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    m.save(outfile = mapped_feature + '_map.html')

In [82]:
create_map(geo_agg, 'ZIP', 'count')

In [593]:
# code before helper function was created

# la_geo = r'updated-file.json'
# m = folium.Map(location = [34.0522, -118.2437], zoom_start = 11)
# m.choropleth(
#     geo_data = la_geo,
#     fill_opacity = 0.7,
#     line_opacity=0.2,
#     data = geo_agg,
#     key_on = 'feature.properties.name',
#     columns = ['facility_zip','average_score'],
#     fill_color='RdYlGn',
#     legend_name='Average Inspection Score',
# )
# folium.LayerControl().add_to(m)
# m.save(outfile='average_score_map.html')

In [187]:
# code before helper function was created

tx_geo = r'updated-tx-file.json'
m2 = folium.Map(location = [27.8006, -97.3964], zoom_start = 10)
m2.choropleth(
     geo_data = tx_geo,
     fill_opacity = 0.7,
     line_opacity=0.2,
     data = geo_agg,
     key_on = 'feature.properties.ZCTA5CE10',
     columns = ['ZIP','count'],
     fill_color='RdYlGn',
     legend_name='Number of Apps in Each Texas Zip Code',
)
folium.LayerControl().add_to(m2)
m2.save(outfile='tx_apps_map.html')


In [186]:
# lets drop the highest zip codes and have a better look at what is left
# if the highest zip codes are in 79xxx and not 78xxx something is messed up

# Lets start with TEXAS, use just texas
tx_inspect_df = inspect_df.query('STATE == "TX"')
tx_zip_counts = tx_inspect_df['ZIP'].value_counts().rename_axis('zip').reset_index(name='count')
print(tx_zip_counts)

# lets drop the top 9 zip codes in CC
tx_zip_counts_filtered = tx_zip_counts.sort_values('count')
tx_zip_counts_filtered = tx_zip_counts_filtered.drop(tx_zip_counts_filtered.tail(9).index)
print(tx_zip_counts_filtered)

tx_geo = r'updated-tx-file.json'
m3 = folium.Map(location = [31.1352, -99.3351], zoom_start = 6)
m3.choropleth(
     geo_data = tx_geo,
     fill_opacity = 0.7,
     line_opacity=0.2,
     data = tx_zip_counts_filtered,
     key_on = 'feature.properties.ZCTA5CE10',
     columns = ['zip','count'],
     fill_color='RdYlGn',
     legend_name='Number of Apps in Each Texas Zip Code minus CC',
)
folium.LayerControl().add_to(m3)
m3.save(outfile='tx-minus-CC_apps_map.html')

        zip  count
0     78412    747
1     78414    687
2     78413    443
3     78418    309
4     78411    194
...     ...    ...
999   78057      1
1000  78934      1
1001  78214      1
1002  75932      1
1003  75567      1

[1004 rows x 2 columns]
        zip  count
1003  75567      1
807   76798      1
808   79916      1
809   78535      1
810   79124      1
...     ...    ...
14    78258     52
13    78380     52
11    78254     56
10    77494     60
9     78363     61

[995 rows x 2 columns]


In [136]:
#
# CREATE US-WIDE GEO data
#

# Lets compare states other than texas
us_inspect_df = inspect_df.query('STATE != "TX"')
us_zip_counts = us_inspect_df['ZIP'].value_counts().rename_axis('zip').reset_index(name='count')
print(us_zip_counts)

# load GeoJSON
with open('zcta5socrata.geo.json', 'r') as jsonFile:
    data = json.load(jsonFile)
tmp = data

# remove ZIP codes not in our dataset
geozips = []
for i in range(len(tmp['features'])):
#    if tmp['features'][i]['properties']['ZCTA5CE10'] in list(us_zip_counts['zip'].unique()):
    if tmp['features'][i]['properties']['zcta5ce10'] in list(us_zip_counts['zip'].unique()):
        geozips.append(tmp['features'][i])

# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("updated-us-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)


        zip  count
0     50014      5
1      7306      4
2     73071      4
3     83201      3
4     80537      3
...     ...    ...
1214  92865      1
1215  92056      1
1216  11422      1
1217  40390      1
1218  19401      1

[1219 rows x 2 columns]


27475915

In [184]:
#
# Create MAP
#

# lets drop the top 9 zip codes in CC
#tx_zip_counts_filtered = tx_zip_counts.sort_values('count')
#tx_zip_counts_filtered = tx_zip_counts_filtered.drop(tx_zip_counts_filtered.tail(9).index)
#print(tx_zip_counts_filtered)

us_geo = r'updated-us-file.json'
m4 = folium.Map(location = [37.1352, -99.3351], zoom_start = 5)
m4.choropleth(
     geo_data = us_geo,
     fill_opacity = 0.7,
     line_opacity=0.2,
     data = us_zip_counts,
     key_on = 'feature.properties.zcta5ce10',
     columns = ['zip','count'],
     fill_color='RdYlGn',
     legend_name='Number of Apps in Each US Zip Code minus TX',
)
folium.LayerControl().add_to(m4)
m4.save(outfile='US_apps_map.html')

In [171]:
# Lets compare states other than texas
# using recoded full state name
us_states_inspect_df = inspect_df.query('STATE != "TX"')
us_state_counts = us_states_inspect_df['STATE_FULL'].value_counts().rename_axis('state').reset_index(name='count')
print(us_state_counts)

#
# Create MAP
#

us_states = r'us-states.geo.json'
m5 = folium.Map(location = [53.58, -110.46], zoom_start = 4)
m5.choropleth(
     geo_data = us_states,
     fill_opacity = 0.7,
     line_opacity=0.2,
     data = us_state_counts,
     key_on = 'feature.properties.name',
     columns = ['state','count'],
     fill_color='RdYlGn',
     legend_name='Number of Apps in Each US State minus Texas',
)
folium.LayerControl().add_to(m5)
m5.save(outfile='US_states_apps_map.html')

             state  count
0       California    224
1          Florida    101
2         New York     72
3          Georgia     60
4         Virginia     59
5         Illinois     50
6         Oklahoma     45
7     Pennsylvania     44
8   North Carolina     44
9       New Jersey     40
10         Arizona     39
11      Washington     36
12       Louisiana     33
13        Maryland     31
14        Missouri     31
15       Tennessee     30
16        Colorado     30
17            Ohio     29
18         Indiana     27
19      New Mexico     27
20  South Carolina     25
21        Arkansas     25
22         Alabama     24
23        Michigan     23
24     Mississippi     21
25            Iowa     19
26     Connecticut     18
27            Utah     18
28       Minnesota     17
29   Massachusetts     17
30       Wisconsin     13
31          Nevada     13
32        Kentucky     13
33          Kansas     13
34          Hawaii     11
35          Oregon     10
36           Maine      8
37          

In [595]:
fac_grouped = inspect_vio_df.groupby(['facility_zip','facility_id'])
fac_agg = fac_grouped.agg({'F044': np.mean,
                        'F033': np.mean,
                        'F035': np.mean,
                        'F036': np.mean,
                        'F040': np.mean,
                        'F043': np.mean,
                        'F037': np.mean,
                        'F039': np.mean,
                        'F030': np.mean,
                        'F006': np.mean,
                        'F014': np.mean,
                        'F007': np.mean})

In [596]:
fac_agg.head(10)

F044  F033  F035  F036  F040  F043  F037  F039  \
facility_zip facility_id                                                   
90001        FA0002183     6.0   3.0   4.0   1.0   1.0   3.0   5.0   0.0   
             FA0002627     1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
             FA0004443     1.0   1.0   1.0   0.0   0.0   1.0   0.0   0.0   
             FA0004465     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
             FA0004529     3.0   1.0   0.0   0.0   0.0   2.0   0.0   0.0   
             FA0004739     6.0   3.0   2.0   0.0   1.0   3.0   2.0   1.0   
             FA0004873     1.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   
             FA0005313     4.0   0.0   2.0   2.0   1.0   4.0   0.0   0.0   
             FA0005764     7.0   3.0   4.0   0.0   2.0   5.0   1.0   3.0   
             FA0005770     3.0   1.0   3.0   3.0   3.0   2.0   0.0   0.0   

                          F030  F006  F014  F007  
facility_zip facility_id                          
90001        FA0002183     2.0   1.0   0.0   2.0  
             FA0002627     0.0   0.0   0.0   0.0  
             FA0004443     0.0   0.0   0.0   0.0  
             FA0004465     0.0   0.0   0.0   0.0  
             FA0004529     0.0   0.0   0.0   0.0  
             FA0004739     2.0   2.0   1.0   0.0  
             FA0004873     0.0   0.0   0.0   0.0  
             FA0005313     0.0   2.0   0.0   0.0  
             FA0005764     4.0   2.0   1.0   1.0  
             FA0005770     1.0   1.0   1.0   0.0

In [597]:
fac_agg = fac_agg.groupby(level=0).mean()
fac_agg.reset_index(inplace=True)

In [598]:
fac_agg.head()

,facility_zip,F044,F033,F035,F036,F040,F043,F037,F039,F030,F006,F014,F007
0,90001,1.717391,1.257246,1.184783,0.655797,0.637681,1.054348,0.782609,0.398551,0.492754,1.090580,0.387681,0.554348
1,90002,1.724638,1.318841,0.753623,0.434783,0.492754,0.971014,0.536232,0.318841,0.434783,0.942029,0.188406,0.376812
2,90003,1.728261,1.695652,1.282609,0.666667,0.670290,1.387681,0.630435,0.369565,0.663043,1.112319,0.380435,0.684783
3,90004,2.729242,2.649819,1.888087,1.595668,1.169675,1.472924,0.794224,0.631769,1.000000,0.953069,0.985560,0.541516
4,90005,2.940397,2.986755,2.125828,1.635762,1.745033,2.099338,1.463576,2.105960,1.480132,1.122517,1.354305,0.880795


In [599]:
def count_distribution(df,location,subgroup):
    group_counts = pd.DataFrame(df.groupby([location,subgroup]).size().unstack(1))
    group_counts.reset_index(inplace = True)
    return group_counts

def subgroup_distribution(df,location,subgroup):
    group = df.groupby([location,subgroup]).size()
    group_pcts = group.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
    group_pcts = pd.DataFrame(group_pcts.unstack(1))
    group_pcts.reset_index(inplace=True)
    return group_pcts

In [600]:
seats_counts = count_distribution(inspect_vio_df, 'facility_zip', 'seats')
est_type_counts = count_distribution(inspect_vio_df, 'facility_zip', 'est_type')
risk_counts = count_distribution(inspect_vio_df, 'facility_zip', 'risk')

In [601]:
# before helper function was written

# seat_counts = pd.DataFrame(inspect_vio_df.groupby(['facility_zip', 'seats']).size().unstack(1))
# seat_counts.reset_index(inplace=True)
# seat_counts.head()

In [602]:
# risk_group = inspect_vio_df.groupby(['facility_zip','risk']).size()

In [603]:
# risk_pcts = risk_group.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))

In [604]:
# risk_pcts = pd.DataFrame(risk_pcts.unstack(1))
# risk_pcts.reset_index(inplace=True)
# risk_pcts

In [605]:
seats_dist = subgroup_distribution(inspect_vio_df, 'facility_zip', 'seats')
est_type_dist = subgroup_distribution(inspect_vio_df, 'facility_zip', 'est_type')
risk_dist = subgroup_distribution(inspect_vio_df, 'facility_zip', 'risk')

In [606]:
seats_dist.head()

seats,facility_zip,0-30,151 +,"2,000+ SF","25-1,999 SF",31-60,61-150
0,90001,45.259042,2.346041,8.308895,30.009775,12.805474,1.270772
1,90002,32.894737,2.631579,14.473684,41.228070,3.947368,4.824561
2,90003,46.936656,NaN,5.815161,38.733126,5.399792,3.115265
3,90004,36.737926,1.979414,6.571655,11.955661,24.386382,18.368963
4,90005,27.384196,5.858311,4.019074,12.602180,31.471390,18.664850


In [607]:
create_map(seats_counts, 'facility_zip', '0-30', 'Occupancy Facility Total Count')
create_map(seats_counts, 'facility_zip', '2,000+ SF', 'Occupancy Facility Total Count')

In [620]:
create_map(seats_dist, 'facility_zip', '0-30', 'Occupancy Concentration')
create_map(seats_dist, 'facility_zip', '2,000+ SF', 'Occupancy Concentration')

In [626]:
create_map(seats_counts, 'facility_zip', '0-30', 'Occupancy Count')
create_map(seats_counts, 'facility_zip', '31-60', 'Occupancy Count')
create_map(seats_counts, 'facility_zip', '61-150', 'Occupancy Count')

In [608]:
est_type_dist.head()

est_type,facility_zip,FOOD MKT RETAIL,RESTAURANT
0,90001,38.318671,61.681329
1,90002,55.701754,44.298246
2,90003,44.548287,55.451713
3,90004,18.527316,81.472684
4,90005,16.621253,83.378747


In [609]:
create_map(est_type_counts, 'facility_zip', 'FOOD MKT RETAIL', ' Total Count')
create_map(est_type_counts, 'facility_zip', 'RESTAURANT', ' Total Count')

In [610]:
risk_dist.head()

risk,facility_zip,HIGH RISK,LOW RISK,MODERATE RISK
0,90001,60.410557,19.550342,20.039101
1,90002,41.228070,28.947368,29.824561
2,90003,49.428868,24.195223,26.375909
3,90004,69.358670,12.430721,18.210610
4,90005,72.207084,11.989101,15.803815


In [618]:
create_map(risk_counts, 'facility_zip', 'LOW RISK', ' Facilities Total Count')
create_map(risk_counts, 'facility_zip', 'HIGH RISK', ' Facilities Total Count')

In [617]:
est_type_counts.head()

est_type,facility_zip,FOOD MKT RETAIL,RESTAURANT
0,90001,392.0,631.0
1,90002,127.0,101.0
2,90003,429.0,534.0
3,90004,234.0,1029.0
4,90005,244.0,1224.0


In [654]:
create_map(fac_agg, 'facility_zip', 'F044', 'Average Violation of Floor, Walls, and Ceiling Maintenance and Condition')
create_map(fac_agg, 'facility_zip', 'F033', 'Average Violation of Nonfood Equipment Cleanliness and Maintenance')
create_map(fac_agg, 'facility_zip', 'F035', 'Average Violation of Equiptment/Utensils Cleanliness and Maintenance')
create_map(fac_agg, 'facility_zip', 'F036', 'Average Violation of Equiptment/Utensils/Linens Storage and Use')
create_map(fac_agg, 'facility_zip', 'F040', 'Average Violation of Plumbing')
create_map(fac_agg, 'facility_zip', 'F014', 'Average Violation of Surface Cleaning and Sanitization')

In [656]:
violation_dict['F014']

'# 14. Food contact surfaces: clean and sanitized'